# Format Spec Mini-Language

This tutorial provides a deep dive into the format spec syntax for t-prompts.

The format spec follows the pattern:
```
{expression:key:render_hints}
```

Where:
- **expression**: The Python expression to interpolate
- **key**: The key for accessing this interpolation (defaults to expression name)
- **render_hints**: Optional metadata for custom rendering (everything after the second `:`)

In [1]:
from t_prompts import prompt

## Basic Format Spec: Key Only

The simplest format spec just specifies a key.

In [2]:
# No format spec: key defaults to the expression
user_query = "What is Python?"
p1 = prompt(t"{user_query}")
print(f"No format spec - Key: {list(p1.keys())[0]}")
print(f"Has 'user_query' key: {'user_query' in p1}")

# With custom key
p2 = prompt(t"{user_query:query}")
print(f"\nWith key - Key: {list(p2.keys())[0]}")
print(f"Has 'query' key: {'query' in p2}")
print(f"Has 'user_query' key: {'user_query' in p2}")

No format spec - Key: user_query
Has 'user_query' key: True

With key - Key: query
Has 'query' key: True
Has 'user_query' key: False


## Format Spec with Render Hints

Add render hints after the key to provide metadata for custom renderers.

In [3]:
# With key and render hints
user_query = "What is Python?"
p3 = prompt(t"{user_query:query:format=json}")

print(f"Key: {list(p3.keys())[0]}")
print(f"Render hints: {p3['query'].render_hints}")
print(f"Value: {p3['query']}")

Key: query
Render hints: format=json
Value: TextInterpolation(key='query', expression='user_query', conversion=None, format_spec='query:format=json', render_hints='format=json', value='What is Python?', index=1)


## Render Hints Are Strings

Render hints are stored as strings and you can parse them however you like.

In [4]:
# Multiple render hints separated by commas
data = '{"name": "Alice"}'
p = prompt(t"{data:user_data:format=json,indent=2}")

node = p['user_data']
print(f"Key: {node.key}")
print(f"Render hints (raw): {node.render_hints!r}")
print(f"Value: {node}")

Key: user_data
Render hints (raw): 'format=json,indent=2'
Value: TextInterpolation(key='user_data', expression='data', conversion=None, format_spec='user_data:format=json,indent=2', render_hints='format=json,indent=2', value='{"name": "Alice"}', index=1)


## Parsing Render Hints

You can implement custom parsers for render hints based on your needs.

In [5]:
def parse_hints(hints_str):
    """Parse render hints as comma-separated key=value pairs."""
    if not hints_str:
        return {}
    result = {}
    for pair in hints_str.split(','):
        if '=' in pair:
            key, value = pair.split('=', 1)
            result[key.strip()] = value.strip()
        else:
            # Handle flags without values
            result[pair.strip()] = True
    return result

# Example with multiple hints
content = "<html><body>Hello</body></html>"
p = prompt(t"{content:html:format=xml,indent=4,preserve_whitespace=true}")

hints = parse_hints(p['html'].render_hints)
print("Parsed hints:")
for key, value in hints.items():
    print(f"  {key}: {value}")

Parsed hints:
  format: xml
  indent: 4
  preserve_whitespace: true


## Built-in Render Hints: sep

The `sep=<value>` render hint is recognized by the default renderer for list interpolations.

In [6]:
# Create list items
item_names = ["apple", "banana", "cherry"]
items = [
    prompt(t"{item_names[i]:item}")
    for i in range(len(item_names))
]

# Default separator (newline)
p1 = prompt(t"{items:items}")
print("Default separator (newline):")
print(str(p1))

# Custom separator
items2 = [
    prompt(t"{item_names[i]:item}")
    for i in range(len(item_names))
]
p2 = prompt(t"{items2:items:sep=, }")
print("\nCustom separator ', ':")
print(str(p2))

Default separator (newline):
apple
banana
cherry

Custom separator ', ':
apple, banana, cherry


## Built-in Render Hints: xml

The `xml=<tag>` render hint wraps content in XML tags.

In [7]:
content = "This is my reasoning process."
p = prompt(t"{content:reasoning:xml=thinking}")

print("With xml=thinking hint:")
print(str(p))
print(f"\nRender hints: {p['reasoning'].render_hints}")

With xml=thinking hint:
<thinking>
This is my reasoning process.
</thinking>

Render hints: xml=thinking


## Built-in Render Hints: header

The `header=<text>` or `header` render hint adds Markdown headers.

In [8]:
task = "Translate the following text"

# Header with explicit text
p1 = prompt(t"{task:t:header=Task Description}")
print("With header=Task Description:")
print(str(p1))

# Header using key as text
p2 = prompt(t"{task:Instructions:header}")
print("\nWith header (using key):")
print(str(p2))

With header=Task Description:
# Task Description
Translate the following text

With header (using key):
# Instructions
Translate the following text


## Combining Multiple Render Hints

You can combine multiple built-in hints together.

In [9]:
reasoning = "First, I'll analyze. Then, I'll conclude."
p = prompt(t"{reasoning:r:header=Analysis:xml=thinking}")

print("Combined header and xml hints:")
print(str(p))
print(f"\nRender hints: {p['r'].render_hints}")

Combined header and xml hints:
# Analysis
<thinking>
First, I'll analyze. Then, I'll conclude.
</thinking>

Render hints: header=Analysis:xml=thinking


## Dynamic Format Specs

Format specs can contain interpolated values to generate keys programmatically.

In [10]:
# Generate keys programmatically
prefix = "config"
section = "database"
value = "postgresql://localhost"

p = prompt(t"{value: {prefix}_{section} }")
print(f"Generated key: {list(p.keys())[0]}")
print(f"Value: {p['config_database']}")

Generated key: config_database
Value: TextInterpolation(key='config_database', expression='value', conversion=None, format_spec=' config_database ', render_hints='', value='postgresql://localhost', index=1)


## Dynamic Keys with Render Hints

Combine dynamic keys with render hints for maximum flexibility.

In [11]:
# Build a prompt with dynamic keys and custom render hints
section = "user_profile"
field = "bio"
content = "Software engineer interested in AI."

p = prompt(t"{content: {section}_{field} :max_length=100,truncate=true}")

node = p['user_profile_bio']
print(f"Key: {node.key}")
print(f"Render hints: {node.render_hints}")
print(f"Value: {node}")

# Parse the hints
hints = parse_hints(node.render_hints)
print("\nParsed hints:")
for k, v in hints.items():
    print(f"  {k}: {v}")

Key: user_profile_bio
Render hints: max_length=100,truncate=true
Value: TextInterpolation(key='user_profile_bio', expression='content', conversion=None, format_spec=' user_profile_bio :max_length=100,truncate=true', render_hints='max_length=100,truncate=true', value='Software engineer interested in AI.', index=1)

Parsed hints:
  max_length: 100
  truncate: true


## Numbered Keys for Lists

Use dynamic keys to create numbered example keys programmatically.

In [12]:
# Create numbered keys
examples = ["First example", "Second example", "Third example"]

example_prompts = [
    prompt(t"{examples[i]: example_{str(i)} }")
    for i in range(len(examples))
]

# List all the keys
print("Generated keys:")
for ep in example_prompts:
    print(f"  {list(ep.keys())[0]}")

Generated keys:
  example_0
  example_1
  example_2


## Accessing Render Hints Programmatically

Build custom tools that leverage render hints for specialized behavior.

In [13]:
def apply_truncation(value, max_length):
    """Truncate a value if it exceeds max_length."""
    if len(value) <= max_length:
        return value
    return value[:max_length-3] + "..."

def custom_renderer(p):
    """Custom renderer that respects truncate hints."""
    parts = []
    for key in p.keys():
        node = p[key]
        value = str(node)

        # Check for truncate hint
        if node.render_hints:
            hints = parse_hints(node.render_hints)
            if 'truncate' in hints and 'max_length' in hints:
                max_len = int(hints['max_length'])
                value = apply_truncation(value, max_len)

        parts.append(value)

    return " ".join(parts)

# Example usage
long_text = "This is a very long piece of text that should be truncated according to the max_length hint."
short_text = "Short."

p = prompt(t"{long_text:long:max_length=30,truncate=true} {short_text:short}")

print("Default rendering:")
print(str(p))
print("\nCustom rendering with truncation:")
print(custom_renderer(p))

Default rendering:
This is a very long piece of text that should be truncated according to the max_length hint. Short.

Custom rendering with truncation:
TextInterpolation(key='long... TextInterpolation(key='short', expression='short_text', conversion=None, format_spec='short', render_hints='', value='Short.', index=3)


## Complex Example: Structured Configuration

Use format specs to build structured, self-documenting configurations.

In [14]:
# Configuration values
app_name = "MyApp"
db_host = "localhost"
db_port = "5432"
api_key = "sk-abc123def456"

# Build configuration with metadata
config = prompt(t"""
Application: {app_name:app_name:type=string,required=true}
Database Host: {db_host:db_host:type=string,default=localhost}
Database Port: {db_port:db_port:type=int,default=5432}
API Key: {api_key:api_key:type=secret,mask=true}
""")

# Display configuration
print("Configuration:")
print(str(config))

# Inspect metadata
print("\nMetadata for each field:")
for key in config.keys():
    node = config[key]
    hints = parse_hints(node.render_hints) if node.render_hints else {}
    print(f"  {key}: {hints}")

Configuration:
Application: MyApp
Database Host: localhost
Database Port: 5432
API Key: sk-abc123def456

Metadata for each field:
  app_name: {'type': 'string', 'required': 'true'}
  db_host: {'type': 'string', 'default': 'localhost'}
  db_port: {'type': 'int', 'default': '5432'}
  api_key: {'type': 'secret', 'mask': 'true'}


## Summary

The format spec mini-language provides powerful control over prompt structure:

✅ **Key specification** - Custom keys for accessing interpolations  
✅ **Render hints** - Metadata for custom rendering behavior  
✅ **Built-in hints** - `sep=`, `xml=`, `header=` recognized by default renderer  
✅ **Custom hints** - Parse render hints however you need  
✅ **Dynamic keys** - Generate keys programmatically with interpolation  
✅ **Flexibility** - Combine all features for complex use cases  

**Pattern:**
```python
{expression:key:render_hints}
```

**Examples:**
- `{value:key}` - Basic key
- `{value:key:xml=tag}` - XML wrapper
- `{value:key:header=Title}` - Markdown header
- `{value:key:custom=hint,other=hint}` - Custom hints
- `{value:{prefix}_{suffix}}` - Dynamic key